## Project - Airline AI Assistant

In [96]:
import os
import json
from dotenv import load_dotenv
import google.generativeai as genai
import gradio as gr

In [97]:
load_dotenv()

google_api_key = os.getenv('GEMINI_API_KEY')

In [98]:
genai.configure(api_key=google_api_key)

In [99]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [100]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [101]:
price_function = genai.protos.FunctionDeclaration(
    name="get_ticket_price",
    description="Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "destination_city": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="The city that the customer wants to travel to"
            )
        },
        required=["destination_city"]
    )
)

In [102]:
def stream_gemini_with_history(message, history):
    
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message,
        tools=[price_function]  
    )
    
    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {message}\nAssistant:"
    
    response = model.generate_content(prompt)
    print(f"Response: {response}")
    
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        function_name = function_call.name
        function_args = dict(function_call.args)
        
        print(f"Function called: {function_name} with args: {function_args}")
        
        if function_name == "get_ticket_price":
            function_result = get_ticket_price(function_args["destination_city"])
            
            new_prompt = prompt + f"\n[Function call: {function_name}({function_args}) returned: {function_result}]\nAssistant:"
            
            final_response = model.generate_content(new_prompt)
            return final_response.text
    
    return response.text
    

In [105]:
stream_gemini_with_history("what is the price to london", [])

Response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_ticket_price",
                  "args": {
                    "destination_city": "london"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.00021342842632697689
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 113,
        "candidates_token_count": 9,
        "total_token_count": 122
      },
      "model_version": "gemini-2.0-flash"
    }),
)
Function called: get_ticket_price with args: {'destination_city': 'london'}
Tool get_ticket_price called for london


'The price of a return ticket to London is $799.\n'